In [1]:
import logging, os
logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf

from pathlib import Path

import keras_tuner as kt

from importlib import reload
import hyper_model_creator
import keras_model_creator
import sentinel_utils

In [2]:
trials_dir = Path('trials', 'hyperband_resnet')
trial_metric = 'val_weightedf2score'

In [3]:
callbacks = [
    tf.keras.callbacks.TensorBoard(trials_dir.joinpath('board')),
    tf.keras.callbacks.EarlyStopping(
        monitor=trial_metric, patience=20, mode='max'
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor=trial_metric, factor=0.5, patience=8, min_lr=1e-6,
    ),
]

In [ ]:
reload(hyper_model_creator)

hypermodel = hyper_model_creator.BuildHyperModel()

tuner = kt.Hyperband(
    hypermodel,
    objective=kt.Objective(trial_metric, direction='max'),
    directory=trials_dir.parent,
    project_name=trials_dir.name,
    hyperband_iterations=3,
    max_epochs=100,
    overwrite=False,
)
tuner.search(
    verbose=1, 
    callbacks=callbacks
)

Trial 299 Complete [00h 06m 41s]
val_weightedf2score: 0.5804163813591003

Best val_weightedf2score So Far: 0.6896040439605713
Total elapsed time: 6d 01h 14m 39s

Search: Running Trial #300

Value             |Best Value So Far |Hyperparameter
True              |True              |class_weight
2017              |2017_2018_2019    |training_years
l1                |l1l2              |kernel_regularizer
0.5               |0.5               |spatial_dropout
relu              |leaky_relu        |activation
4                 |4                 |pool_size
0.1               |0.1               |dropout
False             |False             |bias_initializer
0.001             |0.01              |learning_rate
binary_focal_cr...|binary_crossent...|loss_function
2                 |34                |tuner/epochs
0                 |12                |tuner/initial_epoch
4                 |4                 |tuner/bracket
0                 |3                 |tuner/round

Epoch 1/2
3550/3550 ━━━━━━━━

In [ ]:
# best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

# best_hp.values['units'] = 128
# best_hp.values['dropout'] = 0.3

# model = tuner.hypermodel.build(best_hp)
# model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

In [ ]:
# tuner.results_summary(num_trials=2)

In [ ]:
# %load_ext tensorboard
# tb_path = str(trials_dir.joinpath('board'))
# %tensorboard --logdir "$tb_path"